In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Set Up Environment

In [ ]:
network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'contracts_20210106025209.pkl'

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://52.53.175.151:9000/",       "nid": 3}}

env = connections[network]

In [ ]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [251]:
contracts = {}

# with open(contracts_file_custom, 'rb') as f:
#     contracts = pkl.load(f)

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': ''},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': ''},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': ''},
 'reserve_fund': {'zip': 'core_contracts/reserve_fund.zip',
  'SCORE': ''},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': ''},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': ''},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': ''},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': ''},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': ''},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': ''},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': ''},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': ''}}

In [ ]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [ ]:
wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")

In [ ]:
# Balanced test wallet
btest_wallet = KeyWallet.load("keystores/balanced_test.json", "xxxx")

### Send ICX to Balanced test wallet

In [ ]:
wallet.get_address()


In [ ]:
test_wallet = KeyWallet.load("./keystores/keystore_test1.json", "test1_Account")
transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(btest_wallet.get_address())\
    .value(1000000 * ICX)\
    .step_limit(1000000) \
    .nid(NID) \
    .nonce(2) \
    .version(3) \
    .build()
signed_transaction = SignedTransaction(transaction, test_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

In [266]:
# Add the contracts. Copy the output from this cell the the one above to save the contract addresses.
# contracts['dividends'] = {'zip': 'core_contracts/dividends.zip', 'SCORE': 'cxceadb6419b92f7170b38b8328dbd8deefb4a1cdb'}
deploy = ['loans', 'staking', 'dividends', 'rewards', 'reserve_fund', 'governance', 'dummy_oracle', 'sicx', 'dex', 'icd', 'bal', 'bwt']
for directory in {"core_contracts", "token_contracts"}:
    with os.scandir(directory) as it:
        for file in it:
            archive_name = directory + "/" + file.name
            if file.is_dir() and file.name in deploy:
                make_archive(archive_name, "zip", directory, file.name)
                contracts[file.name]['zip'] = archive_name + '.zip'

now = datetime.utcnow()
filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)
contracts

saving contracts list as contracts_20210121080712.pkl


{'loans': {'zip': 'core_contracts/loans.zip', 'SCORE': ''},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxf3b22ebbf151aad4f74a8490ab2fe60d2106fa31'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx4aae13bd08fb3fc5e666a6d9b262ef036f9bdc64'},
 'reserve_fund': {'zip': 'core_contracts/reserve_fund.zip',
  'SCORE': 'cx29fa40132bf655a822dcf4c2eafcb406c758e408'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cxf22475bcc888cd6a635f6fae374ee69ffbb04d1a'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx31bb0d42d9667fd6acab1bbebcfa3b916f04a3f3'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxd5af9afcf8cd2b90f1bd4a019004a0b4227ceb18'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx2904e9e14d6e2a386174364f1cd4b693d18dc0b0'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx52201d119a70890b1d5fa79fba836ac03cf3e694'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cxb5

In [264]:
contracts

{'loans': {'zip': 'core_contracts/loans.zip', 'SCORE': ''},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxb4c4546ee6589dbacb01fb9ffb8ec2154ec4cfc8'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cxf3c36fd31f9503ab259837210a881bee3527d374'},
 'reserve_fund': {'zip': 'core_contracts/reserve_fund.zip',
  'SCORE': 'cxf13d10a1d4268540186eff8bab3fe6695f3bd96a'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx84044a7405e61281b7eb3fb68b6c767749bcfd3c'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx90fe61e164501a39892c11a5cb49061e9f3ac0ad'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx7823d03c27af951336d6afc99bbf81d6415d7a1c'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cxb13a7056ff7e212f102f392717177f4664c4a070'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx8869fda354795d69fdfbe8ef8f4e29a456273a21'},
 'icd': {'zip': 'token_contracts/icd.zip', 'SCORE': ''},
 '

In [181]:
contracts

{'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx14c12f31b8ca35198eaf862c45b43c5b75baadef'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxff8c6025fb7eed7197cef6f7fc7993eb2abd7c75'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx2771c23c7a11cb9804d8c03347db955a632f366c'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxbc9fad62310a119a48d1e959d552d9c4b4ce4f03'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx5a5db07a2fb83c8850ae2f67dde041d989af3180'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cx2df2231fe5c090ee3e6135b816f2e487c77b306d'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx200c946c7cc03f330bbf170649bbacb9f2998bbb'},
 'reserve_fund': {'zip': 'core_contracts/reserve_fund.zip',
  'SCORE': 'cxb73bbcebf22424ee523af64216d2c55b4c506443'},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': 'cxaa73e0a90c6199ba79262c8cf16473f94001934c'}}

## Deploy All SCOREs

In [265]:
step_limit = 4000100000

for score, loc in contracts.items():
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .build()

#     step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
    print(score)
    print(res)

contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


...

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


bal
{'txHash': '0x8fd265bc0a2947ab5d401b3ed2482141d952117d889edfa173fd3f5dc045d1f4', 'blockHeight': 657047, 'blockHash': '0x7b2a10fd67d89aecb04f41d280ab7ff7c798b440a315869552940ba18ab4c926', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'stepUsed': 1190299360, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1190299360, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
bwt
{'txHash': '0x04b1ceff3c22890ca6de166da0d8e90103802f8782710e8420d15635a9f3282d', 'blockHeight': 657048, 'blockHash': '0x14b85c660264c25471c979354cf326dc20d6fe8eeea65dd6285398322b85592d', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx407e502b4037a0a6fc158deac86e9c95c25a1a80', 'stepUsed': 1166921760, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1166921760, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

{'loans': {'zip': 'core_contracts/loans.zip', 'SCORE': ''},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxf3b22ebbf151aad4f74a8490ab2fe60d2106fa31'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx4aae13bd08fb3fc5e666a6d9b262ef036f9bdc64'},
 'reserve_fund': {'zip': 'core_contracts/reserve_fund.zip',
  'SCORE': 'cx29fa40132bf655a822dcf4c2eafcb406c758e408'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cxf22475bcc888cd6a635f6fae374ee69ffbb04d1a'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx31bb0d42d9667fd6acab1bbebcfa3b916f04a3f3'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxd5af9afcf8cd2b90f1bd4a019004a0b4227ceb18'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx2904e9e14d6e2a386174364f1cd4b693d18dc0b0'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx52201d119a70890b1d5fa79fba836ac03cf3e694'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cxb5

### Deploy a single SCORE

In [267]:
# Deploy a single SCORE
score = 'bal'
loc = {'zip': 'token_contracts/bal.zip',  'SCORE': ''}
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(loc["zip"]))\
    .build()

step_limit = 10000000000


signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
contracts[score]["SCORE"] = res.get('scoreAddress', '')
res

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...


{'txHash': '0x650c7a3d0b4a25dab7d582b566d1bf4e5d5bb1bf59fcbacfdf5f78b9b18fcaab',
 'blockHeight': 657383,
 'blockHash': '0x2143a055b29c0d9b3350a767cef87a4898c0b35ed91a0dbced1083b6f23f56a8',
 'txIndex': 1,
 'to': 'cx0000000000000000000000000000000000000000',
 'scoreAddress': 'cxb3540492c151599a52f25acd6e679b87df58b1d7',
 'stepUsed': 1191467360,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1191467360,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

## Update a SCORE

In [189]:
contract = 'loans'
update = contracts[contract]['SCORE']
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to('cx14c12f31b8ca35198eaf862c45b43c5b75baadef')\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('./loans.zip'))\
    .build()

# step_limit = icon_service.estimate_step(deploy_transaction) + 100000
step_limit = 3000000000

signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...


{'txHash': '0x121adc3ecaff241aa91253bbd95750fff83f6241677ae190994cc5f05e4f3026',
 'blockHeight': 421314,
 'blockHash': '0x5ea95c46f4f754f595f9240d0f566b22728157e7b859d9255880b0816ad8e754',
 'txIndex': 1,
 'to': 'cx14c12f31b8ca35198eaf862c45b43c5b75baadef',
 'scoreAddress': 'cx14c12f31b8ca35198eaf862c45b43c5b75baadef',
 'stepUsed': 2059971120,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 2059971120,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

## Set parameters

In [ ]:
settings = [{'contract': 'sicx', 'method': 'setAdmin', 'params':{'_admin': contracts['staking']['SCORE']}},
            {'contract': 'icd', 'method': 'setAdmin', 'params':{'_admin': contracts['loans']['SCORE']}},
            {'contract': 'bal', 'method': 'setAdmin', 'params':{'_admin': wallet.get_address()}},
            {'contract': 'loans', 'method': 'setAdmin', 'params':{'_admin': wallet.get_address()}},
            {'contract': 'icd', 'method': 'set_oracle_address', 'params':{'_address': contracts['dummy_oracle']['SCORE']}},
            {'contract': 'staking', 'method': 'setSicxAddress', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'loans', 'method': 'set_dividends', 'params':{'_address': contracts['dividends']['SCORE']}},
            {'contract': 'loans', 'method': 'set_reserve', 'params':{'_address': contracts['reserve_fund']['SCORE']}},
            {'contract': 'reserve_fund', 'method': 'set_loans_score', 'params':{'_address': contracts['loans']['SCORE']}},
            {'contract': 'reserve_fund', 'method': 'set_baln_token', 'params':{'_address': contracts['bal']['SCORE']}},
            {'contract': 'reserve_fund', 'method': 'set_sicx_token', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'sicx', 'method': 'set_staking_address', 'params':{'_address': contracts['staking']['SCORE']}},
            {'contract': 'bal', 'method': 'mint', 'params':{'_amount': 10 * ICX}},
            {'contract': 'bal', 'method': 'mintTo', 'params':{'_account': contracts['reserve_fund']['SCORE'], '_amount': 1000 * ICX}},
            {'contract': 'loans', 'method': 'add_asset', 'params':{'_token_address': contracts['sicx']['SCORE'], 'is_active': 1, 'is_collateral': 1}},
            {'contract': 'loans', 'method': 'add_asset', 'params':{'_token_address': contracts['icd']['SCORE'], 'is_active': 1, 'is_collateral': 0}},
            {'contract': 'loans', 'method': 'add_asset', 'params':{'_token_address': contracts['bal']['SCORE'], 'is_active': 0, 'is_collateral': 1}},
            {'contract': 'loans', 'method': 'toggle_loans_on', 'params':{}}]

In [ ]:
for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))

## Run through read-only methods on all SCOREs

### A single read-only method

In [ ]:
params = {}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['staking']['SCORE'])\
                    .method('getSicxAddress')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result
# int(result, 0)

In [ ]:
ro_methods = {"core": {
                 "dex": [{"method": "name", "params": {}},
                         {"method": "get_reference_data", "params": {"base": str, "quote": str}},
                         {"method": "getSicxAddress", "params": {}},
                         {"method": "getDeposit", "params": {"_tokenAddress": str, "_user": str}},
                         {"method": "getPoolId", "params": {"_token1Address": str, "_token2Address": str}},
                         {"method": "getNonce", "params": {}},
                         {"method": "getPoolTotal", "params": {"_pid": int, "_token": str}},
                         {"method": "totalSupply", "params": {"_pid": int}},
                         {"method": "balanceOf", "params": {"_owner": str, "_id": int}},
                         {"method": "balanceOfBatch", "params": {"_owners": List[str], "_ids": List[int]}},
                         {"method": "getFees", "params": {}},
                         {"method": "getPoolBase", "params": {"_pid": int}},
                         {"method": "getPoolQuote", "params": {"_pid": int}},
                         {"method": "getPrice", "params": {"_pid": int}},
                         {"method": "getInversePrice", "params": {"_pid": int}},
                         {"method": "getICXBalance", "params": {}},
                         {"method": "getICXWithdrawLock", "params": {}}]
                 "dividends": [{"method": "name", "params": {}},
                               {"method": "getLoansScore", "params": {}},
                               {"method": "getBalances", "params": {}}],
                 "governance": [{"method": "name", "params": {}}],
                 "loans": [{"method": "name", "params": {}},
                           {"method": "getDividends", "params": {}},
                           {"method": "getReserve", "params": {}},
                           {"method": "getOriginationFee", "params": {}},
                           {"method": "getReplayBatchSize", "params": {}},
                           {"method": "getRedeemMinimum", "params": {}},
                           {"method": "getAssetTokens", "params": {}},
                           {"method": "getCollateralTokens", "params": {}},
                           {"method": "getTotalCollateral", "params": {}},
                           {"method": "getAccountPositions", "params": {"_owner": str}},
                           {"method": "getAvailableAssets", "params": {}},
                           {"method": "assetCount", "params": {}}],
                 "reserve_fund": [{"method": "name", "params": {}},
                                  {"method": "getLoansScore", "params": {}},
                                  {"method": "getBalnToken", "params": {}},
                                  {"method": "getSicxToken", "params": {}},
                                  {"method": "getBalances", "params": {}}],
                 "rewards": [{"method": "name", "params": {}},
                             {"method": "get_id", "params": {}},
                             {"method": "get_rewards", "params": {}},
                             {"method": "get_reward_data", "params": {"_reward": str}}],
                 "staking": [{"method": "name", "params": {}},
                             {"method": "getTodayRate", "params": {}},
                             {"method": "getSicxAddress", "params": {}},
                             {"method": "getTotalStake", "params": {}},
                             {"method": "getLifetimeReward", "params": {}},
                             {"method": "getTopPreps", "params": {}},
                             {"method": "getUserUnstakeInfo", "params": {}},
                             {"method": "getUnstakeAmount", "params": {}}]},
              "token": {
                 "bal": [{"method": "name", "params": {}},
                         {"method": "symbol", "params": {}},
                         {"method": "decimals", "params": {}},
                         {"method": "totalSupply", "params": {}},
                         {"method": "balanceOf", "params": {"_owner": str}},
                         {"method": "getAdmin", "params": {}},
                         {"method": "getPeg", "params": {}},
                         {"method": "getOracle", "params": {}},
                         {"method": "getMinInterval", "params": {}},
                         {"method": "lastPriceInLoop", "params": {}}],
                 "bwt": [{"method": "name", "params": {}},
                         {"method": "symbol", "params": {}},
                         {"method": "decimals", "params": {}},
                         {"method": "totalSupply", "params": {}},
                         {"method": "balanceOf", "params": {"_owner": str}},
                         {"method": "getAdmin", "params": {}}],
                 "icd": [{"method": "name", "params": {}},
                         {"method": "symbol", "params": {}},
                         {"method": "decimals", "params": {}},
                         {"method": "totalSupply", "params": {}},
                         {"method": "balanceOf", "params": {"_owner": str}},
                         {"method": "getAdmin", "params": {}},
                         {"method": "getPeg", "params": {}},
                         {"method": "getOracle", "params": {}},
                         {"method": "getMinInterval", "params": {}},
                         {"method": "lastPriceInLoop", "params": {}}],
                 "sicx": [{"method": "name", "params": {}},
                          {"method": "symbol", "params": {}},
                          {"method": "decimals", "params": {}},
                          {"method": "totalSupply", "params": {}},
                          {"method": "balanceOf", "params": {"_owner": str}},
                          {"method": "getAdmin", "params": {}},
                          {"method": "getPeg", "params": {}},
                          {"method": "getOracle", "params": {}},
                          {"method": "lastPriceInLoop", "params": {}}]
                 }
             }

In [196]:
with open('ro_methods.json', 'r') as f:
    ro_methods = json.load(f)

In [197]:
ro_methods

{'core': {'dex': [{'method': 'name', 'params': {}},
   {'method': 'getReferenceData', 'params': {'base': 'str', 'quote': 'str'}},
   {'method': 'getSicxAddress', 'params': {}},
   {'method': 'getDeposit',
    'params': {'_tokenAddress': 'str', '_user': 'str'}},
   {'method': 'getPoolId',
    'params': {'_token1Address': 'str', '_token2Address': 'str'}},
   {'method': 'getNonce', 'params': {}},
   {'method': 'getPoolTotal', 'params': {'_pid': 'int', '_token': 'str'}},
   {'method': 'totalSupply', 'params': {'_pid': 'int'}},
   {'method': 'balanceOf', 'params': {'_owner': 'str', '_id': 'int'}},
   {'method': 'balanceOfBatch',
    'params': {'_owners': 'List[str]', '_ids': 'List[int]'}},
   {'method': 'getFees', 'params': {}},
   {'method': 'getPoolBase', 'params': {'_pid': 'int'}},
   {'method': 'getPoolQuote', 'params': {'_pid': 'int'}},
   {'method': 'getPrice', 'params': {'_pid': 'int'}},
   {'method': 'getInversePrice', 'params': {'_pid': 'int'}},
   {'method': 'getICXBalance', 'para

In [200]:

import requests
import json
from hashlib import sha3_256
BLOCK = 28693695
TX_HASH = '0x053ba8fc46b2519a084e5d3fa1b3d8c31b358e35389be4f646c2f86f64ce22ab'

rpc_dict = {
            'jsonrpc': '2.0',
            'method': 'icx_getTransactionByHash',
            'id': 1234,
            'params': {'txHash': TX_HASH}
           }
r = requests.post("https://ctz.solidwallet.io/api/v3", 
                  data=json.dumps(rpc_dict))
int(json.loads(r.text)['result']['timestamp'], 0) / 86400000000

18628.622629322395

In [201]:
24 * .622629322395

14.943103737480001

In [ ]:
ro_methods = {"loans": ["name", "getAvailableAssets", "assetCount", "getCollateralTokens", "getTotalCollateral"],
              "staking": ["name", "getSicxAddress", "getTodayRate"],
              "sicx": ["name", "symbol", "decimals", "totalSupply", "getAdmin", "lastPriceInLoop"],
              "icd": ["name", "symbol", "decimals", "totalSupply", "getAdmin"],
              "bal": ["name", "symbol", "decimals", "totalSupply", "getAdmin"]
              }

In [180]:
results = {}
for score in ro_methods.keys():
    print()
    print(f'SCORE: {score}')
    print(f'Address: {contracts[score]["SCORE"]}')
    results[score] = {}
    for method in ro_methods[score]:
        call = CallBuilder().from_(wallet.get_address())\
                            .to(contracts[score]['SCORE'])\
                            .method(method)\
                            .build()
        result = icon_service.call(call)
        if method in ['totalSupply', 'decimals', 'get_total_collateral', 'get_rate']:
            result = int(result, 0)
            if method in ['totalSupply', 'get_total_collateral', 'get_rate']:
                result = result / ICX
        print(f'{method}: {result}')
        results[score][method] = result


SCORE: loans
Address: cx14c12f31b8ca35198eaf862c45b43c5b75baadef
name: BalancedLoans
getAvailableAssets: {'sICX': {'symbol': 'sICX', 'address': 'cxbc9fad62310a119a48d1e959d552d9c4b4ce4f03', 'peg': 'sICX', 'added': '0x5b8e733047cf2', 'is_collateral': '0x1', 'active': '0x1'}, 'ICD': {'symbol': 'ICD', 'address': 'cx2df2231fe5c090ee3e6135b816f2e487c77b306d', 'peg': 'USD', 'added': '0x5b8e7332306ae', 'is_collateral': '0x0', 'active': '0x1', 'bad_debt': '0xad78ebc5ac6200000', 'liquidation_pool': '0x248f1c2fa4e3087a56096329820000000'}}
assetCount: 0x3
getCollateralTokens: {'sICX': 'cxbc9fad62310a119a48d1e959d552d9c4b4ce4f03', 'BALN': 'cxaa73e0a90c6199ba79262c8cf16473f94001934c'}
getTotalCollateral: 0x5abb68ab2b980f9f7e

SCORE: staking
Address: cxff8c6025fb7eed7197cef6f7fc7993eb2abd7c75
name: Staking
getSicxAddress: cxbc9fad62310a119a48d1e959d552d9c4b4ce4f03
getTodayRate: 0xde2c055fdd24880

SCORE: sicx
Address: cxbc9fad62310a119a48d1e959d552d9c4b4ce4f03
name: StakedICX
symbol: sICX
decimals: 

## Deposit ICX and Mint ICD Loan

In [191]:
data = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_asset\": \"ICD\", \"_amount\": 40000000000000000000}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(800 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
{'txHash': '0x4a16fc84e7a3e8365546ad70e99f16de89a9ceb8d8b4634861390ecc285fd53c', 'blockHeight': 421580, 'blockHash': '0x75a30e244d93d226afecf1eb7a32b427d71c07860e0b3577184170329cea573a', 'txIndex': 1, 'to': 'cxff8c6025fb7eed7197cef6f7fc7993eb2abd7c75', 'stepUsed': 820980, 'stepPrice': 10000000000, 'cumulativeStepUsed': 820980, 'eventLogs': [{'scoreAddress': 'cx2df2231fe5c090ee3e6135b816f2e487c77b306d', 'indexed': ['OraclePrice(str,int,int)', 'ICXUSD', '0x2454d8dcb2cf44bb', '0x5b8f6a87fcfb3'], 'data': []}, {'scoreAddress': 'cx2df2231fe5c090ee3e6135b816f2e487c77b306d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx2df2231fe5c090ee3e6135b816f2e487c77b306d', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x22b1c8c1227a00000'], 'data': ['0x4e6f6e65']}, {'scoreAddress': 'cx2df2231fe5c090ee3e6135b816f2e487c77b306d', 'indexed': ['Mint(Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x22b1c8c1227a00000', '0x4e6f6e65']}, {'scoreAddre

### Just deposit collateral

In [192]:
data = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_asset\": \"\", \"_amount\": 0}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(30 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
{'txHash': '0xfd35c77dfcf6ae91bf871fefcbeedff5b240cd38c0335b4b11f2d7fb8ce206cd', 'blockHeight': 421596, 'blockHash': '0xe438e7fa5a0c983e7717480e9a7cb82c963315203dcddbd2b37ffcfb3ac46ee6', 'txIndex': 1, 'to': 'cxff8c6025fb7eed7197cef6f7fc7993eb2abd7c75', 'stepUsed': 441140, 'stepPrice': 10000000000, 'cumulativeStepUsed': 441140, 'eventLogs': [{'scoreAddress': 'cxbc9fad62310a119a48d1e959d552d9c4b4ce4f03', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxbc9fad62310a119a48d1e959d552d9c4b4ce4f03', 'cx14c12f31b8ca35198eaf862c45b43c5b75baadef', '0x1a006d8763e3a24e6'], 'data': ['0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72726f77222c2022706172616d73223a207b225f6173736574223a2022222c20225f616d6f756e74223a20307d7d']}, {'scoreAddress': 'cxbc9fad62310a119a48d1e959d552d9c4b4ce4f03', 'indexed': ['Mint(Address,int,bytes)', 'cx14c12f31b8ca35198eaf862c45b43c5b75baadef'], 'data': ['0x1a006d8763e3a24e6', '0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f7

### Repay Some ICD

In [ ]:
data = "{\"method\": \"_repay_loan\", \"params\": {}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_value': 5 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

### Withdraw Some Collateral

In [ ]:
params = {'_value': 5 * ICX}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("withdraw_collateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

### Test Liquidation

In [ ]:
# Deposit collateral to fresh wallet
data = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_asset\": \"\", \"_amount\": 0}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_data': data}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(782769 * ICX // 1000)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

#### Update the loans SCORE to add a loan of ICD to the Balanced test wallet
Code in the on_update() method will mint and add the ICD. Adds an undercollateralized amount of ICD, 200 ICD.

In [ ]:
params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

In [ ]:
params = {'_position': btest_wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("liquidate")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

In [ ]:
params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

In [ ]:
params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['icd']['SCORE'])\
                    .method('balanceOf')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
int(result, 0) // 10**18